# Process `cosmosis` chains

Credit @ Otavio Alves for sharing his script. The full `python` script can be found [here](https://github.com/cosmodesi/desi-y1-kp7/blob/main/plotting/plot_cosmosis.py).

In [2]:
import numpy as np

import getdist

import argparse
import configparser

## Otavio's routine to read and process `cosmosis` chain into `GetDist`-friendly format below

In [3]:
# Label dictionary: from cosmosis names to latex
label_dict = {
             'cosmological_parameters--omega_m': r'\Omega_m',
             'cosmological_parameters--omega_c': r'\Omega_c',
             'cosmological_parameters--omega_k': r'\Omega_k',
             'cosmological_parameters--omega_lambda': r'\Omega_\Lambda',
             'cosmological_parameters--ommh2': r'\Omega_m h^2',
             'cosmological_parameters--ommh3': r'\Omega_m h^3',
             'cosmological_parameters--ombh2': r'\Omega_b h^2',
             'cosmological_parameters--omch2': r'\Omega_c h^2',
             'cosmological_parameters--h0':      r'h',
             'cosmological_parameters--omega_b': r'\Omega_b',
             'cosmological_parameters--n_s':     r'n_s',
             'cosmological_parameters--a_s':     r'A_s',
             'cosmological_parameters--omnuh2':  r'\Omega_{\nu}',
             'cosmological_parameters--sigma_8': r'\sigma_8',
             'cosmological_parameters--sigma_12': r'\sigma_{12}',
             'cosmological_parameters--s8': r'S_8',
             'cosmological_parameters--w': r'w',
             'cosmological_parameters--wa': r'w_a',
             'cosmological_parameters--w0_fld':  r'w_{GDM}',
             'cosmological_parameters--w_gdm':  r'w_{GDM}',
             'cosmological_parameters--cs2_gdm': r'c_s^2',
             'cosmological_parameters--cv2_gdm': r'c_v^2',
             'cosmological_parameters--log_cs2_gdm': r'\log c_s^2',
             'cosmological_parameters--log_cv2_gdm': r'\log c_v^2',
             'cosmological_parameters--log_cs2': r'log(c_s^2)',
             'intrinsic_alignment_parameters--a': r'A_{IA}',
             'intrinsic_alignment_parameters--alpha': r'\alpha_{IA}',
             'bin_bias--b1': 'b_1',
             'bin_bias--b2': 'b_2',
             'bin_bias--b3': 'b_3',
             'bin_bias--b4': 'b_4',
             'bin_bias--b5': 'b_5',
             'shear_calibration_parameters--m1': 'm_1',
             'shear_calibration_parameters--m2': 'm_2',
             'shear_calibration_parameters--m3': 'm_3',
             'shear_calibration_parameters--m4': 'm_4',
             'lens_photoz_errors--bias_1': r'\Delta z^{lens}_1',
             'lens_photoz_errors--bias_2': r'\Delta z^{lens}_2',
             'lens_photoz_errors--bias_3': r'\Delta z^{lens}_3',
             'lens_photoz_errors--bias_4': r'\Delta z^{lens}_4',
             'lens_photoz_errors--bias_5': r'\Delta z^{lens}_5',
             'wl_photoz_errors--bias_1': r'\Delta z^{source}_1',
             'wl_photoz_errors--bias_2': r'\Delta z^{source}_2',
             'wl_photoz_errors--bias_3': r'\Delta z^{source}_3',
             'wl_photoz_errors--bias_4': r'\Delta z^{source}_4',
             'wl_photoz_errors--bias_5': r'\Delta z^{source}_r',
             'bias_lens--b1': 'b_1',
             'bias_lens--b2': 'b_2',
             'bias_lens--b3': 'b_3',
             'bias_lens--b4': 'b_4',
             'bias_lens--b5': 'b_5',
             'intrinsic_alignment_parameters--a1': 'A_1',
             'intrinsic_alignment_parameters--a2': 'A_2',
             'intrinsic_alignment_parameters--alpha1': '\\alpha_1',
             'intrinsic_alignment_parameters--alpha2': '\\alpha_2',
             'intrinsic_alignment_parameters--bias_ta': 'b_{ta}',
             'supernova_params--m': 'M',
              }


def load_chain(filename, burn=0):
    """loads chain file dropping data_vector columns"""
    """better than numpy.loadtxt if you have data_vector columns"""
    data = []
    with open(filename) as f:
        labels = np.array(f.readline()[1:-1].lower().split())
        mask = ["data_vector" not in l for l in labels]
        for line in f.readlines():
            if '#' in line:
                continue
            else:
                data.append(np.array(line.split(), dtype=np.double)[mask])
    data = {labels[mask][i]: col for i,col in enumerate(np.array(data)[burn:,:].T)}
    return data

def load_ini(filename, section=None):
    """loads given ini file. If section is given, loads that section from a cosmosis chain file"""
    values = configparser.ConfigParser(strict=False)

    if section is None:
        values.read_file(filename)
    else:
        section = section.upper()
        with open(filename) as f:
            line = f.readline()
            lines=[]
            while("START_OF_{}".format(section) not in line):
                line = f.readline()

            while("END_OF_{}".format(section) not in line):
                line = f.readline()
                lines.append(line.replace('#', ''))
            values.read_string('\r'.join(lines[:-1]))
    return values

def get_ranges(values_ini, params2plot):
    """loads range values from values.ini file object"""

    return {p:
            (lambda x: [float(x[0]), float(x[2])] if len(x) == 3 else [None, None])(values_ini.get(*p.split('--')).split()) \
            if values_ini.has_option(*p.split('--')) \
            else [None, None] \
            for p in params2plot}

def get_fiducial(values_ini, params, extra={}):
    """loads fiducial values from values.ini file object"""

    fiducial = {p:
            float((lambda x: x[1] if len(x) == 3 else x[0])(values_ini.get(*p.split('--')).split())) \
            if values_ini.has_option(*p.split('--')) \
            else None \
            for p in params}
    
    add_S8(fiducial)

    for k in extra.keys():
        fiducial[k] = extra[k]

    return fiducial

def get_sampler(params_ini):
    """reads the sampler name from a given chain"""
    return params_ini.get('runtime', 'sampler')

def get_nlive(params_ini):
    """reads the number of live points from a given chain"""
    nlive = params_ini.get(get_sampler(params_ini), 'live_points')
    return int(nlive)

# Get label from list of cosmosis' parameter names
get_label = np.vectorize(lambda label: label_dict[label] \
        if label in label_dict.keys() else label)

def on_params(arr, params2plot):
    """returns array of data from parameter list"""
    return np.array([arr[l] for l in params2plot]).T

def add_S8(data):
    """computes S8"""
    if 'cosmological_parameters--sigma_8' in data:
        data['cosmological_parameters--s8'] = data['cosmological_parameters--sigma_8']*(data['cosmological_parameters--omega_m']/0.3)**0.5
    return data

def add_omxh2(data):
    """computes physical densities"""
    if 'cosmological_parameters--omega_m' in data and 'cosmological_parameters--h0' in data and 'cosmological_parameters--omega_b' in data:
        data['cosmological_parameters--ommh2'] = data['cosmological_parameters--omega_m']*data['cosmological_parameters--h0']**2
        data['cosmological_parameters--ommh3'] = data['cosmological_parameters--omega_m']*data['cosmological_parameters--h0']**3
        data['cosmological_parameters--ombh2'] = data['cosmological_parameters--omega_b']*data['cosmological_parameters--h0']**2
        data['cosmological_parameters--omch2'] = data['cosmological_parameters--ommh2'] - data['cosmological_parameters--ombh2']
        data['cosmological_parameters--omega_c'] = data['cosmological_parameters--omega_m'] - data['cosmological_parameters--omega_b']
    return data

def get_all_params(data):
    """returns all parameters from a given chain"""
    return [k for k in data.keys() if k not in ['weight', 'like', 'post', 'prior', 'data_vector--2pt_chi2']]

def loadMCSamples(filename):

    data = load_chain(filename)
    add_S8(data)
    add_omxh2(data)

    params = get_all_params(data)

    # print(f"ESS: {sum(data['weight'])**2/sum(data['weight']**2)}")

    values_ini = load_ini(filename, section='values')
    params_ini = load_ini(filename, section='params')
    
    return getdist.MCSamples(
                samples=on_params(data, params),
                weights=data['weight'] if 'weight' in data.keys() else None,
                loglikes=data['like'],
                ranges=get_ranges(values_ini, params),
                sampler='nested' if get_sampler(params_ini) in ['multinest', 'polychord'] else 'mcmc',
                names=params,
                labels=[l for l in get_label(params)],
            )


## Load, process and save new output chain

In [27]:
#input_des_chain='/nfs/turbo/lsa-nguyenmn/DESI_kp7/DES_data/chain_3x2pt_lcdm_SR_maglim.txt'
input_des_chain = # the path to your input chain
input_des_samples=loadMCSamples(input_des_chain)

Removed no burn in


In [28]:
#output_des_chain='/nfs/turbo/lsa-nguyenmn/DESI_kp7/DES_data/chain_3x2pt_lcdm_SR_maglim_GetDist'
output_des_chain = # the path to your output chain
input_des_samples.saveAsText(output_des_chain)